In [2]:
import numpy as np
import torch
import gym
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
from environment import *
from mix_state_env import MixStateEnv
from config import *
import copy
from MyGlobal import MyGlobals
from itertools import count
from torch.distributions import Categorical
import random
import math
from collections import namedtuple, deque
import pickle





In [3]:
with open('D:/Lab/RL/AODAI_RL/code/dqn_train18_statedict.pkl', 'rb') as file:
    old_model_statedict = pickle.load(file)
old_model_statedict


OrderedDict([('layer1.weight',
              tensor([[-0.0208, -0.0109,  0.1083,  ..., -0.0147, -0.1367,  0.0177],
                      [ 0.1321,  0.0342,  0.0463,  ...,  0.0644,  0.0334, -0.1071],
                      [ 0.1157, -0.0842,  0.0430,  ...,  0.0191,  0.0538,  0.0979],
                      ...,
                      [-0.1276,  0.0628,  0.0183,  ..., -0.1086, -0.1444,  0.0941],
                      [ 0.0909, -0.0237,  0.0379,  ...,  0.0808,  0.0339,  0.0542],
                      [ 0.0741,  0.0913, -0.0917,  ..., -0.0683,  0.1017,  0.0341]])),
             ('layer1.bias',
              tensor([ 0.1624, -0.1937, -0.0747, -0.0064, -0.0516,  0.2141, -0.0795, -0.0710,
                      -0.0170,  0.0192, -0.0393,  0.1741,  0.0314,  0.1211, -0.0605, -0.0147,
                      -0.1384,  0.1963,  0.0811, -0.1166,  0.0890,  0.0602, -0.1061,  0.1139,
                       0.1459, -0.0561, -0.1492, -0.1537,  0.0298,  0.1224,  0.0936,  0.1346,
                      -0.0486,

In [4]:
weight_layer1=old_model_statedict['layer1.weight']
new_weight_layer1_dis=torch.zeros(128)
new_weight_layer1_wait=torch.zeros(128)
for i in range(5):
    new_weight_layer1_dis+=weight_layer1[:,4+(i*2)]
    new_weight_layer1_wait+=weight_layer1[:,5+(i*2)]
new_weight_layer1_dis/=5
new_weight_layer1_wait/=5
print(weight_layer1)
new_weight_layer1= torch.cat((weight_layer1, new_weight_layer1_dis.unsqueeze(1)), dim=1)

new_weight_layer1= torch.cat((new_weight_layer1, new_weight_layer1_wait.unsqueeze(1)), dim=1)
print(new_weight_layer1)
old_model_statedict['layer1.weight']=new_weight_layer1

tensor([[-0.0208, -0.0109,  0.1083,  ..., -0.0147, -0.1367,  0.0177],
        [ 0.1321,  0.0342,  0.0463,  ...,  0.0644,  0.0334, -0.1071],
        [ 0.1157, -0.0842,  0.0430,  ...,  0.0191,  0.0538,  0.0979],
        ...,
        [-0.1276,  0.0628,  0.0183,  ..., -0.1086, -0.1444,  0.0941],
        [ 0.0909, -0.0237,  0.0379,  ...,  0.0808,  0.0339,  0.0542],
        [ 0.0741,  0.0913, -0.0917,  ..., -0.0683,  0.1017,  0.0341]])
tensor([[-0.0208, -0.0109,  0.1083,  ...,  0.0177, -0.0381, -0.0318],
        [ 0.1321,  0.0342,  0.0463,  ..., -0.1071,  0.0413,  0.0389],
        [ 0.1157, -0.0842,  0.0430,  ...,  0.0979,  0.0291,  0.0346],
        ...,
        [-0.1276,  0.0628,  0.0183,  ...,  0.0941, -0.0453, -0.0011],
        [ 0.0909, -0.0237,  0.0379,  ...,  0.0542,  0.0338,  0.0185],
        [ 0.0741,  0.0913, -0.0917,  ...,  0.0341,  0.0938, -0.0335]])


In [5]:
weight_layer3=old_model_statedict['layer4.weight']
new_weight_layer3=torch.zeros(128)
for i in range(5):
    new_weight_layer3+=weight_layer3[i+2]
    
new_weight_layer3/=5
new_weight_layer3=new_weight_layer3.unsqueeze(0)
print(weight_layer3)
print(new_weight_layer3)
new_weight3=torch.cat((weight_layer3, new_weight_layer3), dim=0)


print(new_weight3)
old_model_statedict['layer4.weight']=new_weight3

tensor([[ 2.2168e-02,  2.8930e-03, -9.5783e-02,  2.8657e-02,  2.8579e-02,
         -1.5545e-01,  3.6584e-03,  3.3648e-02,  1.8507e-02,  2.4250e-02,
          1.3956e-02, -6.5999e-02, -1.6132e-01,  5.3293e-02, -7.9664e-02,
         -3.7987e-02, -6.6297e-02,  5.2369e-02,  8.8079e-02, -1.5360e-01,
          3.4321e-03,  2.2678e-02, -7.7790e-02, -1.4479e-03, -1.9072e-02,
          7.3816e-02, -8.5363e-02, -5.1918e-02,  5.3969e-02, -6.1089e-03,
          8.1789e-03,  7.6159e-03,  1.5331e-02,  8.5412e-02,  2.2728e-02,
         -8.1031e-02, -5.2842e-02, -1.5856e-02,  8.9385e-04, -3.9922e-03,
          5.0160e-02, -2.9404e-02, -1.6381e-01,  9.6885e-03,  2.0512e-02,
         -1.9972e-02, -6.1902e-02,  2.8536e-02,  8.7092e-02, -3.8820e-02,
          2.1239e-02, -2.1476e-02, -9.6382e-03, -2.9223e-02,  2.7177e-04,
          2.4709e-02,  6.0575e-02,  2.8465e-02, -6.3071e-02, -1.5622e-01,
          1.5215e-02, -6.8532e-02, -3.3467e-02,  2.2098e-02, -2.5348e-03,
         -1.2244e-02, -8.1215e-02,  6.

In [6]:
bias_layer3=old_model_statedict['layer4.bias']
new_bias_layer3=torch.zeros(1)
for i in range(5):
    new_bias_layer3+=bias_layer3[i+2]
new_bias_layer3/=5
new_bias_layer3=new_bias_layer3
bias_layer3=torch.cat((bias_layer3,new_bias_layer3))
old_model_statedict['layer4.bias']=bias_layer3


In [ ]:
with open('dqn_new_statedict2.pkl', 'wb') as file:
    pickle.dump(old_model_statedict, file)

In [3]:
with open('D:/Lab/RL/AODAI_RL/code/dqn_new_statedict2.pkl', 'rb') as file:
    new_model_statedict = pickle.load(file)

In [4]:
new_model_statedict

OrderedDict([('layer1.weight',
              tensor([[-0.0208, -0.0109,  0.1083,  ...,  0.0177, -0.0381, -0.0318],
                      [ 0.1321,  0.0342,  0.0463,  ..., -0.1071,  0.0413,  0.0389],
                      [ 0.1157, -0.0842,  0.0430,  ...,  0.0979,  0.0291,  0.0346],
                      ...,
                      [-0.1276,  0.0628,  0.0183,  ...,  0.0941, -0.0453, -0.0011],
                      [ 0.0909, -0.0237,  0.0379,  ...,  0.0542,  0.0338,  0.0185],
                      [ 0.0741,  0.0913, -0.0917,  ...,  0.0341,  0.0938, -0.0335]])),
             ('layer1.bias',
              tensor([ 0.1624, -0.1937, -0.0747, -0.0064, -0.0516,  0.2141, -0.0795, -0.0710,
                      -0.0170,  0.0192, -0.0393,  0.1741,  0.0314,  0.1211, -0.0605, -0.0147,
                      -0.1384,  0.1963,  0.0811, -0.1166,  0.0890,  0.0602, -0.1061,  0.1139,
                       0.1459, -0.0561, -0.1492, -0.1537,  0.0298,  0.1224,  0.0936,  0.1346,
                      -0.0486,

In [5]:
class DQNnet(nn.Module):
    def __init__(self,n_observations,n_actions):
        super(DQNnet,self).__init__()
        self.layer1=nn.Linear(n_observations,128)
        self.layer2=nn.Linear(128,256)
        self.layer3=nn.Linear(256,128)
        self.layer4=nn.Linear(128,n_actions)
    def forward(self,x):
        x=F.relu(self.layer1(x))
        x=F.relu(self.layer2(x))
        x=F.relu(self.layer3(x))
        return self.layer4(x)
policy_net=DQNnet(14,7)
optimizer=optim.AdamW(filter(lambda p: p.requires_grad, policy_net.parameters()),lr=1e-4,amsgrad=True)


In [9]:
for name, param in policy_net.named_parameters():
    print(name,param)

layer1.weight Parameter containing:
tensor([[-0.0070,  0.1401,  0.2464,  ..., -0.1052,  0.0129, -0.2311],
        [ 0.1071, -0.0229,  0.2617,  ...,  0.2350, -0.1032, -0.1057],
        [-0.2354, -0.2294, -0.0012,  ..., -0.0440, -0.2575,  0.0715],
        ...,
        [-0.0690, -0.1373, -0.0456,  ..., -0.1778,  0.0527, -0.2369],
        [ 0.1328,  0.0661,  0.1029,  ...,  0.1365,  0.0132,  0.0300],
        [ 0.2045, -0.0098,  0.0525,  ..., -0.1210, -0.0261,  0.0015]],
       requires_grad=True)
layer1.bias Parameter containing:
tensor([-0.0980, -0.1844,  0.0213, -0.2642,  0.0603,  0.1832,  0.0318,  0.1357,
         0.1050, -0.2270,  0.2054, -0.1187, -0.1879,  0.1804, -0.2532, -0.1962,
        -0.2069,  0.0883,  0.1741,  0.1938, -0.0830, -0.2096,  0.0520,  0.0971,
        -0.1173, -0.2059,  0.1572, -0.1079, -0.0476, -0.0325, -0.2203,  0.0399,
         0.2270, -0.1221, -0.2238, -0.0487,  0.1712,  0.0515, -0.0369,  0.2567,
         0.0996, -0.2283, -0.0564, -0.1093, -0.0643,  0.1138,  0.2622

In [10]:
policy_net.named_parameters()

TypeError: 'generator' object is not subscriptable